# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road**

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

def slope(line):
    for x1,y1,x2,y2 in line:
        return (y2-y1)/(x2-x1)
    return 0
    
def lane_coordinates(lane, threshHold, lane_type):
    xMax = yMax = xMin = yMin = ""
    for line in lane:
        for x1,y1,x2,y2 in line:
            
            if lane_type == "right":
                if x1 < threshHold and x2 < threshHold:
                    continue
            elif lane_type == "left":
                if x1 > threshHold and x2 > threshHold:
                    continue
            
            if xMax == "":
                xMax = x1
                yMax = y1
                
            if xMin == "":
                xMin = x1
                yMin = y1
            
            xMax = max(x1, x2, xMax)
            yMax = max(y1, y2, yMax)
            
            xMin = min(x1, x2, xMin)
            yMin = min(y1, y2, yMin)                    
    
    return xMin, yMin, xMax, yMax
    
def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    left_lane = []
    right_lane = []
    xArr = []
    yArr = np.array([])
    combinedArr = np.array([])
    
    for index in range(len(lines)):
        line = lines[index]
        slp = slope(line)
        
        if slp > 0.3:
            right_lane.append(line);
        elif slp < -0.3:
            left_lane.append(line);        
         
    imshape = img.shape
    
    threshHold_right = imshape[1]/2 + 10
    xMin_right, yMin_right, xMax_right, yMax_right = lane_coordinates(right_lane, threshHold_right, 'right')
    
    threshHold_left = imshape[1]/2 - 10
    xMin_left, yMin_left, xMax_left, yMax_left = lane_coordinates(left_lane, threshHold_left, 'left')
    
    #print(yMax_left, yMax_right, yMin_left, yMin_right)
    yMax  = max(yMax_left, yMax_right)
    ymin  = max(yMin_left, yMin_right)
    
    c_right = yMax_right - ((yMax_right - yMin_right)/(xMax_right - xMin_right))*xMax_right 
    xMax_right = int((yMax - c_right)/((yMax_right - yMin_right)/(xMax_right - xMin_right)))
    
    c_left = yMax_left - ((yMax_left - yMin_left)/(xMin_left - xMax_left))*xMin_left 
    xMin_left = int((yMax - c_left)/((yMax_left - yMin_left)/(xMin_left - xMax_left)))
    
    # draw right line
    cv2.line(img, (xMin_right, yMin_right), (xMax_right, yMax), color, 8)
    # draw left line
    cv2.line(img, (xMin_left, yMax), (xMax_left, yMin_left), color, 8)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.       
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    `initial_img` should be the image before any processing.
    The result image is computed as follows:
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def dark_img(img, value=80):
    return np.where((img) < value,0,img-value)

def process_image(image):
    # Read in and grayscale the image
    gray_img = grayscale(image)
    darkened_gray_img = dark_img(gray_img)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(darkened_gray_img, kernel_size)

    # Define our parameters for Canny and apply
    low_threshold = 50
    high_threshold = 150
    edges = canny(blur_gray, low_threshold, high_threshold)

    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(460, 320), (imshape[1]-450, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 40 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    line_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)

    # Create a "color" binary image to combine with line image
    color_edges = np.dstack((edges, edges, edges)) 

    # Draw the lines on the edge image
    lines_edges = weighted_img(line_image, image, 0.8, 1, 0) 

    return lines_edges

In [2]:
# Code to process a video

output_file = 'output/solidWhiteRight.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(output_file, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_file))


# Code to process an image 

    # image = mpimg.imread('test_images/solidWhiteRight.jpg')

    # output = process_image(image)
    # plt.imshow(output)

    # fig1 = plt.gcf()
    # fig1.savefig('test_images_output/solidWhiteRight.jpg', dpi=100)

[MoviePy] >>>> Building video output/solidWhiteRight.mp4
[MoviePy] Writing video output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:07<00:00, 28.15it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output/solidWhiteRight.mp4 

CPU times: user 4.03 s, sys: 1.13 s, total: 5.16 s
Wall time: 8.83 s
